In [13]:
!pip install mwclient
!pip install transformers
!pip install torch torchvision torchaudio
!pip install tensorflow






In [14]:
import mwclient
import time
from transformers import pipeline
from statistics import mean
import pandas as pd
from datetime import datetime

In [15]:
site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]
revs = list(page.revisions())
revs = sorted(revs, key=lambda rev: rev["timestamp"])

In [16]:
sentiment_pipeline = pipeline("sentiment-analysis")

def find_sentiment(text):
  sent = sentiment_pipeline([text[:250]])[0]
  score=sent["score"]
  if sent["label"] == "NEGATIVE":
    score *= -1
  return score

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [17]:
edits = {}

for rev in revs:
  date = time.strftime("%Y-%m-%d", rev["timestamp"])

  if date not in edits:
    edits[date] = dict(sentiments=list(), edit_count=0)

  try:
    edits[date]["edit_count"] += 1
    comment = rev["comment"]
    edits[date]["sentiments"].append(find_sentiment(comment))
  except Exception as e:
    continue

In [18]:
for key in edits:
  if len(edits[key]["sentiments"]) > 0:
    edits[key]["sentiment"] = mean(edits[key]["sentiments"])
    edits[key]["neg_sentiment"] = len([s for s in edits[key]["sentiments"] if s < 0]) / len(edits[key]["sentiments"])
  else:
    edits[key]["sentiment"] = 0
    edits[key]["neg_sentiment"] = 0

  del edits[key]["sentiments"]
     

In [19]:
edits_df = pd.DataFrame.from_dict(edits, orient="index")
edits_df.index = pd.to_datetime(edits_df.index)

In [20]:
dates = pd.date_range(start="2009-03-08", end=datetime.today())

edits_df =edits_df.reindex(dates, fill_value=0)
rolling_edits = edits_df.rolling(30).mean()
rolling_edits = rolling_edits.dropna()
rolling_edits.to_csv("wikipedia_edits.csv")

In [21]:
edits_df

,edit_count,sentiment,neg_sentiment
2009-03-08,4,0,0
2009-03-09,0,0,0
2009-03-10,0,0,0
2009-03-11,0,0,0
2009-03-12,0,0,0
...,...,...,...
2024-06-09,0,0,0
2024-06-10,0,0,0
2024-06-11,0,0,0
2024-06-12,0,0,0
